In [1]:
#initialize parametrized variables

experimentName = ''
runID = ''


In [2]:
# Parameters
runID = "f3031cc2b5ef4d2792e350972e9b8b51"
experimentName = "dummyabc"


In [3]:
import jupyter_client
jupyter_client.kernelspec.find_kernel_specs()

{'python3': '/home/pedro/anaconda3/envs/mlflow-4b4b36ea87f6e682b2d3f4c654d8d652083d52cd/share/jupyter/kernels/python3'}

In [4]:
import pandas as pd
import numpy as np

import mlflow, mlflow.sklearn
from mlflow.pyfunc import PythonModel

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [5]:
#sets the MLflow experiment and active run
mlflow.set_experiment(experiment_name=experimentName)
mlflow.start_run(runID)

<ActiveRun: >

In [6]:
#data loading from storage to be handled by project_name
data = pd.read_csv('/home/pedro/Documentos/work/Jupyter_Python/SuperTopicModeling/BERT-service/dataset/smart-intents.csv', header=None, names=['Input','Intent'])
data = data.sample(1500)



In [7]:
class STMPredProba(PythonModel):
    
    def __init__(self, n):
        self._n = n
        self._model = None

    def fit(self, X, y):
        #defining pipe model
        pipeline = Pipeline([('scaler',MinMaxScaler()),
                             ('model', LogisticRegression(C=1,
                                                          solver='saga',
                                                          multi_class='multinomial',
                                                          max_iter=10000))]) 
        #constructing accuracy
        try: #initially can not be done when classes are unitary..
            X_train, X_test, y_train, y_test = train_test_split(X,
                                                                y,
                                                                test_size=0.2,
                                                                stratify = label_series) # to be contructed from previous predictions and respective new labels to avoid retraining
            pipeline.fit(X = X_train, y = y_train) #constructing the model to accuracy -> to be cut out: see above
            acc = pipeline.score(X = X_test,y = y_test)
            print(acc)
            mlflow.log_metric('accuracy',acc)
        except:
            pass
        # training the model to be served
        pipeline.fit(X, y)
        #store the MLflow model 
        mlflow.sklearn.log_model(pipeline,'SupTopModel')
        self._model = pipeline
        return None

    def predict(self, X, n=0):#, pipe_model, n=0):
        
        from vectorizer import vectorize
        
        X['sentence_vector'] = vectorize(X['Input'])
        vector_df = pd.DataFrame(list(X['sentence_vector']), index=data.index)        
        prob_df = pd.DataFrame(self._model.predict_proba(vector_df), index=vector_df.index, columns=self._model.classes_)
        
        if n ==0:
            return prob_df
        else:
            top_n = np.argpartition(prob_df.transpose(), -self._n, axis=0)[-self._n:].apply(lambda x:
                                                                                  prob_df.transpose().index.values[
                                                                                      x]).transpose()
            top_n.columns = ["{}".format(i+1) for i in range(self._n)]
            prob_matrix = pd.DataFrame()
            for col in top_n.columns:
                prob_matrix[col] = prob_df.apply(lambda row: row[top_n[col][row.name]], axis=1)
            top_prob = pd.merge(top_n, prob_matrix, right_index=True, left_index=True)
            tuple_class_prob = pd.DataFrame()
            for i in range(1, self._n+1): #constructs the tuple table
                runner_x = '_'.join([str(i), 'x'])
                runner_y = '_'.join([str(i), 'y'])
                tuple_class_prob[i] = pd.DataFrame([top_prob[runner_x], top_prob[runner_y]]).transpose().apply(tuple,
                                                                                                           axis=1)
            tuple_class_prob.columns = [f"top{i+1}"+str(i) for i in range(1, self._n+1)]
            for i in range(len(tuple_class_prob)): #sorts the tuple table
                list_to_sort=tuple_class_prob.iloc[i].tolist()
                list_to_sort.sort(key=lambda x: -x[1])
                tuple_class_prob.iloc[i]=list_to_sort
            return tuple_class_prob

In [8]:
#mlflow.end_run()

